# SPARK_STREAMING_API

In [33]:
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
import time
import os
os.getcwd()

'/usr/local/hadoop'

## Loading Streaming Data

In [47]:
# Loading streaming data

def remove_quotation(x):
        return ([xx.replace('"','') for xx in x])

def load():
    
    downloadsRDD = ssc.textFileStream("file:///usr/local/hadoop/streaming/")
    downloadsRDD = downloadsRDD.map(lambda x: x.split(','))
    downloadsRDD = downloadsRDD.map(remove_quotation)
    return downloadsRDD  


## Starting, Computing, and Saving Streaming Computations 
    

###  Q1. To calculate the number of downloads of each package.

In [51]:

# To store the Results of Streaming computations in Cassandra
def save_1(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(lambda row: Row(time=time, package=row[0], count=row[1])))
        df.write.format("org.apache.spark.sql.cassandra").options(table="q1", keyspace="streaming").save(mode="append")
    except:
        pass

#Streaming Computation with batch lengh of 5 seconds
ssc = StreamingContext (sc,5)

#Calling function load() to load streaming data
lines=load()

#Streaming computation
down_each_pack = lines.map(lambda x: (x[6],1))
down_each_pack = down_each_pack.reduceByKey(lambda a,b: a+b)

#Saving in Apache Cassandra
down_each_pack.foreachRDD(save_1)

# Start Streaming
ssc.start()
time.sleep(30)
ssc.stop(stopSparkContext=False)


### Q2.To find the top most downloaded package.

In [8]:
# To store the Results of Streaming computations in Cassandra

def save_2(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(lambda row: Row(time=time, top_most_down_package=row[0], count=row[1])))
        df.write.format("org.apache.spark.sql.cassandra").options(table="q2", keyspace="streaming").save(mode="append")
    except:
        pass

#Streaming Computation with batch lengh of 5 seconds
ssc = StreamingContext (sc,5)

#Calling function load() to load streaming data
lines=load()
    
#Streaming computation
top_1 = lines.map(lambda x:(x[6],1))
top_1 = top_1.reduceByKey(lambda a,b: a+b)
top_1_down_pack = top_1.transform(lambda rdd: rdd.sortBy(lambda x: x[1], ascending=False))
top_1_down_pack = top_1_down_pack.transform(lambda rdd:sc.parallelize(rdd.take(1)))

#Saving in Apache Cassandr
top_1_down_pack.foreachRDD(save_2)

# Start Streaming
ssc.start()
time.sleep(50)
ssc.stop(stopSparkContext=False)

### Q3.To find the top 5 countries along with number of downloads.

In [117]:
# Step 1: To store the Results of Streaming computations in Cassandra
def save_3(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(lambda row: Row(time=time, top_5_countries=row[0], count=row[1])))
        df.write.format("org.apache.spark.sql.cassandra").options(table="q3", keyspace="streaming").save(mode="append")
    except:
        pass

# Step 2: Streaming Computation with batch lengh of 5 seconds
ssc = StreamingContext (sc,5)

# Step 3: Calling function load() to load streaming data
lines=load()
    
# Step 4: Streaming computation
top_5 = lines.map(lambda x: (x[8],1))
top_5 = top_5.reduceByKey(lambda a,b: a+b)
top_5_countries = top_5.transform(lambda rdd: rdd.sortBy(lambda x: x[1], ascending=False))
top_5_countries = top_5_countries.transform(lambda rdd:sc.parallelize(rdd.take(5)))

# Step 5: Saving in Apache Cassandra
top_5_countries.foreachRDD(save_3)

# Step 6: Start Streaming
ssc.start()
time.sleep(50)
ssc.stop(stopSparkContext=False)

### Q4. To find number of downloads for ggplot2 package

In [45]:
# Step1: To store the Results of Streaming computations in Cassandra
def save_4(time, rdd):
    try:
        df = spark.createDataFrame(rdd.map(lambda row: Row(time=time, package=row[0], count=row[1])))
        df.write.format("org.apache.spark.sql.cassandra").options(table="q4", keyspace="streaming").save(mode="append")
    except:
        pass

# Step 2: Streaming Computation with batch lengh of 5 seconds
ssc = StreamingContext (sc,5)

# Step 3: Calling function load() to load streaming data

lines=load()

#Step 4: Streaming computation
ggplot2 = lines.map(lambda x: (x[6],1))
ggplot2 = ggplot2.reduceByKey(lambda a,b: a+b)
ggplot2_download = ggplot2.filter(lambda x: x[0]=='ggplot2')

#Step 5: Saving in Apache Cassandra
ggplot2_download.foreachRDD(save_4)

# Step 6: Start Streaming
ssc.start()
time.sleep(50)
ssc.stop(stopSparkContext=False)